ENVIRONMENT SETTING

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [11]:
import pandas as pd

In [12]:
import os
import time
import urllib
from tqdm import tqdm, trange

In [13]:
tqdm.pandas()

In [14]:
options = webdriver.ChromeOptions()
# options.add_argument('no-sandbox')
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument('disable-gpu')

CRAWL REVIEWS

In [15]:
filenames = os.listdir('./data/conferences/')

In [ ]:
for filename in filenames:
    conference_df = pd.read_csv(f'./data/conferences/{filename}', index_col = 0)

    progress = tqdm(total = len(conference_df))
    dfs = []
    accept = 'accept' in filename

    for title, link in conference_df.loc:
        try:
            driver = webdriver.Chrome(service = Service(executable_path = './driver/chromedriver'), options = options)
            driver.get(link)
            time.sleep(5)

            data = {}

            data['Paper Title'] = title
            data['Paper Link'] = link
            data['Accept'] = 'accept' in filename

            metas = driver.find_elements(By.CSS_SELECTOR, 'div.forum-container > div.note > div.note_contents')
            metadata = {meta.find_element(By.CSS_SELECTOR, 'span.note_content_field').text[:-1]: meta.find_element(By.CSS_SELECTOR, 'span.note_content_value').text for meta in metas}
            
            for k, v in metadata.items():
                data[k] = v
            
            reviews = driver.find_elements(By.CSS_SELECTOR, 'div.note_with_children')

            for review in reviews:
                try:
                    # driver.execute_script('arguments[0].scrollIntoView(true);', review)
                    # time.sleep(1)

                    data['Review Title'] = [x.text for x in review.find_elements(By.CSS_SELECTOR, 'div.note.panel > div.title_pdf_row > h2.note_content_title > span')][0]
                    data['Review Signature'] = [x.text for x in review.find_elements(By.CSS_SELECTOR, 'div.note.panel > div.meta_row > span.signatures')][0]
                    data['Review Date'] = [x.text for x in review.find_elements(By.CSS_SELECTOR, 'div.note.panel > div.clearfix > div.meta_row.pull-left > span.date.item')][0]
                    data['Review Meta'] = [x.text for x in review.find_elements(By.CSS_SELECTOR, 'div.note.panel > div.clearfix > div.meta_row.pull-left > span.item:nth-child(2)')][0]
                    data['Review Subtitles'] = [x.text[:-1] for x in review.find_elements(By.CSS_SELECTOR, 'div.note.panel > div.note_contents > span.note_content_field')][:len(review.find_elements(By.CSS_SELECTOR, 'div.note_contents')) - len(review.find_elements(By.CSS_SELECTOR, 'div.children div.note_contents')) + 1]
                    data['Review Contents'] = [x.text for x in review.find_elements(By.CSS_SELECTOR, 'div.note.panel > div.note_contents > span.note_content_value')][:len(review.find_elements(By.CSS_SELECTOR, 'div.note_contents')) - len(review.find_elements(By.CSS_SELECTOR, 'div.children div.note_contents')) + 1]
                    
                    dfs.append(pd.DataFrame({k: pd.Series([v]) for k,v in data.items()}))
                except Exception as e:
                    break
        except Exception as e:
            break
        finally:
            driver.close()
            progress.update(1)
    break

In [8]:
df = pd.concat(dfs).reset_index(drop = True)

In [ ]:
conference_df.loc[0]